<a href="https://colab.research.google.com/github/sreyyoo/youtube-comment-analyser/blob/main/experiment_3_tfidf_(1%2C3)_max_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlflow

In [ ]:
import os
import mlflow

# Replace with your actual username and token
os.environ["MLFLOW_TRACKING_USERNAME"] = "sreyo2004"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "54b0fc2a87e30ce5bfc3a8ed37bac2ef6d65c1ad"

mlflow.set_tracking_uri("https://dagshub.com/sreyo2004/mlflow-test.mlflow")

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Himanshu-1703/reddit-sentiment-analysis/refs/heads/main/data/reddit.csv')
df.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [ ]:
# Drop missing and duplicate values
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df = df[~(df['clean_comment'].str.strip() == '')]

# Import necessary libraries
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Define the preprocessing function
def preprocess_comment(comment):
    # Convert to lowercase
    comment = comment.lower()

    # Remove trailing and leading whitespaces
    comment = comment.strip()

    # Remove newline characters
    comment = re.sub(r'\n', ' ', comment)

    # Remove non-alphanumeric characters, except punctuation
    comment = re.sub(r"[^A-Za-z0-9\s!?.]", '', comment)

    # Remove stopwords but retain important ones for sentiment analysis
    stop_words = set(stopwords.words('english')) - {'not', 'but', 'however', 'no', 'yet'}
    comment = ' '.join([word for word in comment.split() if word not in stop_words])

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    comment = ' '.join([lemmatizer.lemmatize(word) for word in comment.split()])

    return comment

# Apply the preprocessing function to the `clean_comment` column
df['clean_comment'] = df['clean_comment'].apply(preprocess_comment)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import mlflow

mlflow.set_experiment("Exp 3 - TfIdf Trigram max_features")

2025/06/05 16:00:52 INFO mlflow.tracking.fluent: Experiment with name 'Exp 3 - TfIdf Trigram max_features' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/ee533b3d1de345a699a3837cd5590f8c', creation_time=1749139252777, experiment_id='7', last_update_time=1749139252777, lifecycle_stage='active', name='Exp 3 - TfIdf Trigram max_features', tags={}>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def run_experiment_tfidf_max_features(max_features):
  ngram_range=(1,3)

  vectorizer=TfidfVectorizer(ngram_range=ngram_range,max_features=max_features)

  X_train,X_test,y_train,y_test=train_test_split(df['clean_comment'],df['category'],test_size=0.2,random_state=42)

  X_train=vectorizer.fit_transform(X_train)
  X_test=vectorizer.transform(X_test)

  with mlflow.start_run() as run:

    mlflow.set_tag("mlflow.runName",f"tfidf_max_features_{max_features}")
    mlflow.set_tag("experiment_type","feature engineering")
    mlflow.set_tag("model_type","RandomForestClassifier")

    mlflow.set_tag("description",f"RandomForestClassifier with TF-IDF ngram_range={ngram_range} and max_features={max_features}")

    mlflow.log_param("ngram_range",ngram_range)
    mlflow.log_param("vectorizer_type", "TF-IDF")
    mlflow.log_param("max_features",max_features)

    n_estimators = 200
    max_depth = 15

    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
    y_pred = model.predict(X_test)

        # Log accuracy
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)


    classification_report=classification_report(y_test,y_pred,output_dict=True)
    for label,metrics in classification_report.items():
     if isinstance(metrics,dict):
      for metric,value in metrics.items():
        mlflow.log_metric(f"classification_report_{label}_{metric}",value)


     conf_matrix=confusion_matrix(y_test,y_pred)
     plt.figure(figsize=(8,6))
     sns.heatmap(conf_matrix,annot=True,fmt='d',cmap="Blues")
     plt.xlabel("Predicted")
     plt.ylabel("Actual")
     plt.title(f"Confusion  Matrix: TF-IDF Trigrams, max_features={max_features}")
     plt.savefig("confusion_matrix.png")
     mlflow.log_artifact("confusion_matrix.png")
     plt.close()


     mlflow.sklearn.log_model(model,f"randomforest_model_tfidf_max_features_{max_features}")

max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)





🏃 View run tfidf_max_features_1000 at: https://dagshub.com/sreyo2004/mlflow-test.mlflow/#/experiments/7/runs/45502c1c523a4291bff03c24b0f6f30b
🧪 View experiment at: https://dagshub.com/sreyo2004/mlflow-test.mlflow/#/experiments/7


UnboundLocalError: cannot access local variable 'classification_report' where it is not associated with a value

In [ ]:
def run_experiment_tfidf_max_features(max_features):
  ngram_range=(1,3)

  vectorizer=TfidfVectorizer(ngram_range=ngram_range,max_features=max_features)

  X_train,X_test,y_train,y_test=train_test_split(df['clean_comment'],df['category'],test_size=0.2,random_state=42)

  X_train=vectorizer.fit_transform(X_train)
  X_test=vectorizer.transform(X_test)

  with mlflow.start_run() as run:

    mlflow.set_tag("mlflow.runName",f"tfidf_max_features_{max_features}")
    mlflow.set_tag("experiment_type","feature engineering")
    mlflow.set_tag("model_type","RandomForestClassifier")

    mlflow.set_tag("description",f"RandomForestClassifier with TF-IDF ngram_range={ngram_range} and max_features={max_features}")

    mlflow.log_param("ngram_range",ngram_range)
    mlflow.log_param("vectorizer_type", "TF-IDF")
    mlflow.log_param("max_features",max_features)

    n_estimators = 200
    max_depth = 15

    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
    y_pred = model.predict(X_test)

        # Log accuracy
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

    # Calculate and log classification report metrics
    # Make sure to assign the result of classification_report to the local variable
    classification_report_dict = classification_report(y_test,y_pred,output_dict=True)
    for label,metrics in classification_report_dict.items():
     if isinstance(metrics,dict):
      for metric,value in metrics.items():
        mlflow.log_metric(f"classification_report_{label}_{metric}",value)

    # Calculate and log confusion matrix (moved outside the classification report loop)
    conf_matrix=confusion_matrix(y_test,y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(conf_matrix,annot=True,fmt='d',cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion  Matrix: TF-IDF Trigrams, max_features={max_features}")
    plt.savefig("confusion_matrix.png")
    mlflow.log_artifact("confusion_matrix.png")
    plt.close()

    mlflow.sklearn.log_model(model,f"randomforest_model_tfidf_max_features_{max_features}")

max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)

2025/06/05 16:46:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run tfidf_max_features_1000 at: https://dagshub.com/sreyo2004/mlflow-test.mlflow/#/experiments/7/runs/49ed481658ab492e952e22e738e78193
🧪 View experiment at: https://dagshub.com/sreyo2004/mlflow-test.mlflow/#/experiments/7


2025/06/05 16:47:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run tfidf_max_features_2000 at: https://dagshub.com/sreyo2004/mlflow-test.mlflow/#/experiments/7/runs/af507d0160e34ea2b2a25efa27081c13
🧪 View experiment at: https://dagshub.com/sreyo2004/mlflow-test.mlflow/#/experiments/7


2025/06/05 16:47:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run tfidf_max_features_8000 at: https://dagshub.com/sreyo2004/mlflow-test.mlflow/#/experiments/7/runs/d2adb7e100c240c3a2838bc7edf7ef2a
🧪 View experiment at: https://dagshub.com/sreyo2004/mlflow-test.mlflow/#/experiments/7


2025/06/05 16:51:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run tfidf_max_features_9000 at: https://dagshub.com/sreyo2004/mlflow-test.mlflow/#/experiments/7/runs/da4c94a986944d98afed3f2f4a96f081
🧪 View experiment at: https://dagshub.com/sreyo2004/mlflow-test.mlflow/#/experiments/7


2025/06/05 16:52:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run tfidf_max_features_10000 at: https://dagshub.com/sreyo2004/mlflow-test.mlflow/#/experiments/7/runs/7cd3aeb59c8d46de8250f7b8d52cb1c5
🧪 View experiment at: https://dagshub.com/sreyo2004/mlflow-test.mlflow/#/experiments/7
